In [102]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
from tensorflow import keras
from tensorflow.keras import layers

def load_data():
    """
    Load the MNIST dataset and normalize the pixel values.

    This function loads the MNIST dataset using Keras's built-in dataset API.
    It normalizes the pixel values of the images by dividing them by 255.0.
    
    Returns:
    - X_train (numpy.ndarray): The training data, a 3D array of shape (num_samples, 28, 28).
    - y_train (numpy.ndarray): The labels for the training data, a 1D array of shape (num_samples,).
    - X_test (numpy.ndarray): The test data, a 3D array of shape (num_samples, 28, 28).
    - y_test (numpy.ndarray): The labels for the test data, a 1D array of shape (num_samples,).
    """
    (X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
    X_train, X_test = X_train / 255.0, X_test / 255.0

    return X_train, y_train, X_test, y_test


def convert_to_tf_dataset(X_train, y_train, X_test, y_test):
    # Convert to TensorFlow Datasets
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(256)

    return train_dataset, test_dataset


class LeNet5(tf.keras.Model):
    """
    LeNet-5 model for image classification.

    Attributes:
    - Layers for the LeNet-5 architecture (convolutional, pooling, dense layers).

    Methods:
    - call: Forward pass for the model.
    - step: Compute and apply gradients for one training batch.
    - train: Train the model on a dataset.
    - set_trainable_variables: Set the trainable variables of the model.
    - trainable_vars_as_vector: Return the trainable variables as a 1D tensor.
    """
    def __init__(self, cnn_input_reshape, num_classes):
        """
        Initialize the LeNet-5 model with given input shape and number of output classes.

        Args:
        - cnn_input_reshape (tuple): The shape to which the input should be reshaped. (e.g., (28, 28, 1))
        - num_classes (int): Number of output classes.
        """
        
        super(LeNet5, self).__init__()
        
        self.reshape = layers.Reshape(cnn_input_reshape)
        
        # Layer 1 Conv2D
        self.conv1 = layers.Conv2D(filters=6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='same')
        # Layer 2 Pooling Layer
        self.avgpool1 = layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')
        # Layer 3 Conv2D
        self.conv2 = layers.Conv2D(filters=16, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid')
        # Layer 4 Pooling Layer
        self.avgpool2 = layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')
        self.flatten = layers.Flatten()
        self.dense1 = layers.Dense(units=120, activation='tanh')
        self.dense2 = layers.Dense(units=84, activation='tanh')
        self.dense3 = layers.Dense(units=num_classes, activation='softmax')

    def call(self, inputs, training=None):
        """
        Forward pass for the model.

        Args:
        - inputs (tf.Tensor): Input tensor (batch of images).
        - training (bool, optional): Whether the forward pass is for training.

        Returns:
        - x (tf.Tensor): Output tensor (batch of class probabilities).
        """
        x = self.reshape(inputs)
        x = self.conv1(x)
        x = self.avgpool1(x)
        x = self.conv2(x)
        x = self.avgpool2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        
        return x

    #@tf.function
    def step(self, batch):
        #print("Retrace")
        """
        Perform one training step on a given batch of data.

        Args:
        - batch (tuple): A tuple containing two elements:
            - x_batch (tf.Tensor): A batch of input data.
            - y_batch (tf.Tensor): A batch of labels.

        This method computes the gradients using backpropagation and updates the model's trainable parameters.
        """
        
        x_batch, y_batch = batch

        with tf.GradientTape() as tape:
            # Forward pass: Compute predictions
            y_batch_pred = self(x_batch, training=True)

            # Compute the loss value
            loss = self.loss(y_batch, y_batch_pred)

        # Compute gradients
        gradients = tape.gradient(loss, self.trainable_variables)
        
        # Apply gradients to the model's trainable variables (update weights)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    def train(self, dataset):
        """
        Train the model on an entire dataset.

        Args:
        - dataset (tf.data.Dataset): The dataset on which the model will be trained. 
        """
        for batch in dataset:
            self.step(batch)

    def set_trainable_variables(self, trainable_vars):
        """
        Set the model's trainable variables.

        Args:
        - trainable_vars (list of tf.Tensor): A list of tensors representing the trainable variables to be set.

        This method sets each of the model's trainable variables to the corresponding tensor in `trainable_vars`.
        """
        for model_var, var in zip(self.trainable_variables, trainable_vars):
            model_var.assign(var)

    def trainable_vars_as_vector(self):
        """
        Get the model's trainable variables as a single vector.

        Returns:
        - tf.Tensor: A 1D tensor containing all of the model's trainable variables.
        """
        return tf.concat([tf.reshape(var, [-1]) for var in self.trainable_variables], axis=0)


def get_compiled_and_built_lenet(cnn_batch_input, cnn_input_reshape, num_classes):
    """
    Compile and build a LeNet-5 model.

    Args:
    - cnn_batch_input (tuple): The shape of the input including batch size (e.g., (None, 28, 28)).
    - cnn_input_reshape (tuple): The shape to which the input should be reshaped (e.g., (28, 28, 1)).
    - num_classes (int): Number of output classes.

    Returns:
    - cnn (LeNet5): A compiled and built LeNet-5 model.
    """
    cnn = LeNet5(cnn_input_reshape, num_classes)
    
    cnn.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),  # we have softmax
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')]
    )
    
    cnn.build(cnn_batch_input)
    
    return cnn


X_train, y_train, X_test, y_test = load_data()
train_dataset, test_dataset = convert_to_tf_dataset(X_train, y_train, X_test, y_test)        
        
lenet = get_compiled_and_built_lenet((None, 28, 28), (28, 28, 1), 10)

batch_size = 256
num_steps_until_rtc_check = 1
shuffle_size = train_dataset.cardinality()  # Uniform shuffling

Num GPUs Available:  0


In [107]:
ds = train_dataset.shuffle(shuffle_size).repeat().batch(batch_size).take(1).prefetch(10)

In [109]:
%%timeit

for _ in range(25):
    for batch in ds:
        lenet.step(batch)

3.86 s ± 16.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [110]:
ds = train_dataset.shuffle(shuffle_size).repeat().batch(batch_size).prefetch(10).take(1)

In [111]:
%%timeit

for _ in range(25):
    for batch in ds:
        lenet.step(batch)

3.91 s ± 41.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
